In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Tutorial2_DataFrames")\
        .getOrCreate()

22/11/17 21:30:43 WARN Utils: Your hostname, bigdata-vmware resolves to a loopback address: 127.0.1.1; using 192.168.10.135 instead (on interface ens33)
22/11/17 21:30:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/17 21:30:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# Shuts down spark session
spark.stop()

In [11]:
#df = spark.read.json("data/people.json")
df = spark.read.csv("data/people.csv", header=True, inferSchema=True)
#df = spark.read.parquet("data/users.parquet")

#df.printSchema()
#df.show()
#df.show(3)
#df.collect()
#df.first()
df.count()

#df.dtypes


5

In [8]:
# Steps to load data from hdfs:
# 1) You should exit VS Code. Why? Because python uses the same port (9000) as hdfs.
# 2) You should start hdfs and yarn and copy your data to hdfs.
# 3) Reopen VS Code.
# 4) Open file /home/bigdata/hadoop-3.4.4/etc/hadoop/core-site.xml and copy the value of property "fs.defaultFS", which represents the Hadoop name node path. In our case, it should have value "hdfs://localhost:9000".
# 5) Start the spark session (first code cell of this notebook).
# 6) Run this code cell.

#df = spark.read.text("hdfs://ocalhlost:9000/user/bigdata/data/book.txt")
df = spark.read.csv("hdfs://localhost:9000/user/bigdata/data/people.csv", header=True)
df.show()

In [12]:
rows = [['Maria', 25, 'Manager'],
        ['João', 30, 'Data Scientist'],
        ['Jorge', 30, 'Developer'],
        ['Rita', 28, 'Data Scientist'],
        ['António', 32, 'Developer']]

columns = ['Name', 'age', 'job']
  
dataframe = spark.createDataFrame(rows, columns)
  
dataframe.show()

+-------+---+--------------+
|   Name|age|           job|
+-------+---+--------------+
|  Maria| 25|       Manager|
|   João| 30|Data Scientist|
|  Jorge| 30|     Developer|
|   Rita| 28|Data Scientist|
|António| 32|     Developer|
+-------+---+--------------+



In [36]:
# Select only the "name" and "age" columns

df1 = spark.read.csv("data/people.csv", header=True)

df2 = df1.select("name", "age")

df2.show()

+-----+---+
| name|age|
+-----+---+
|Jorge| 30|
|  Bob| 32|
+-----+---+



In [5]:
# Select all the columns, but increment the age by 1

df = spark.read.csv("data/people.csv", header=True)

df.select("name", df.age + 1, "job").show()

+-----+---------+--------------+
| name|(age + 1)|           job|
+-----+---------+--------------+
|Maria|     26.0|       Manager|
| João|     31.0|Data Scientist|
|Jorge|     31.0|     Developer|
| Rita|     -1.0|Data Scientist|
|  Bob|     33.0|     Developer|
+-----+---------+--------------+



Know your data

In [14]:
df = spark.read.csv("data/people.csv", header=True, inferSchema=False)

#df.summary().show()

#Showing statitics for the age column only
df.select("age").summary().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 5|
|   mean|              23.0|
| stddev|14.212670403551895|
|    min|                -2|
|    25%|              25.0|
|    50%|              30.0|
|    75%|              30.0|
|    max|                32|
+-------+------------------+



In [15]:
# Getting the (distinct) values of some column

df = spark.read.csv("data/people.csv", header=True)
df.show()

age_distinct_values = df.select('age').distinct().collect()

for row in age_distinct_values:
    print(row[0])

#This shows all values
#from pyspark.sql.functions import *
#all_values = df.select(collect_list("age")).first()[0]
#print(all_values)


+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

30
25
32
-2


In [15]:
# Count people by age

df = spark.read.csv("data/people.csv", header=True)

df = df.groupBy("age").count()

df.show()

# groupBy() returns a GroupData, wich consists in set of methods for aggregations on a DataFrame, created by DataFrame.groupBy().\n",
# The count() method of GroupData counts the number of records for each group and returns a DataFrame."

+---+-----+
|age|count|
+---+-----+
| 30|    2|
| 25|    1|
| 32|    1|
| -2|    1|
+---+-----+



Data Cleansing

In [17]:
# Remove columns

df = spark.read.csv("data/people.csv", header=True)
df.show()

#df2 = df.drop("age")
df2 = df.drop("age", "job")

df2.show()

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

+-----+------+-----+
| name|salary|prize|
+-----+------+-----+
|Maria|  2000|  100|
| João|  1500|  250|
|Jorge|  1400|  200|
| Rita|  1600|  500|
|  Bob|  1000|   -1|
+-----+------+-----+



In [20]:
# Remove rows with NULL values

#Note: in order to try this, remove some value(s) in the dataset

df = spark.read.csv("data/people.csv", header=True) 
df.show()

#df2 = df.dropna() #the same as df.dropna("any")
#df2 = df.na.drop() #the same df2 = df.dropna()
#df2 = df.dropna("all") #removes rows with null values in all columns
df2 = df.dropna(subset=["age","job"]) #removes rows with null values in the given columns

df2.show()

#-------------------

#drop(how='any', thresh=None, subset=None)

#All these parameters are optional.

#how – This takes values ‘any’ or ‘all’. By using ‘any’, drop a row if it contains NULLs on any columns. By using ‘all’, drop a row only if all columns have NULL values. Default is ‘any’.
#thresh – This takes int value, Drop rows that have less than threshhold non-null values. Default is ‘None’.
#subset – Use this to select the columns for NULL values. Default is ‘None.


+-----+----+--------------+------+-----+
| name| age|           job|salary|prize|
+-----+----+--------------+------+-----+
|Maria|null|       Manager|  2000|  100|
|Maria|  25|       Manager|  null|  100|
| null|null|          null|  null| null|
| João|  30|Data Scientist|  1500|  250|
|Jorge|  30|     Developer|  1400|  200|
| Rita|  -2|          null|  1600|  500|
|  Bob|  32|     Developer|  1000|   -1|
| null|  32|     Developer|  1000|   -1|
+-----+----+--------------+------+-----+

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  null|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
|  Bob| 32|     Developer|  1000|   -1|
| null| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+



In [21]:
# Remove duplicate rows

df = spark.read.csv("data/people.csv", header=True)
df.show()
print(df.count())

df2 = df.distinct()
df2.show()
print(df2.count())

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

7
+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| Rita| -2|Data Scientist|  1600|  500|
|Jorge| 30|     Developer|  1400|  200|
| João| 30|Data Scientist|  1500|  250|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

5


In [22]:
# Remove rows that have the same combination of values accross multiple selected columns

df = spark.read.csv("data/people.csv", header=True)
df.show()
print(df.count())

df2 = df.dropDuplicates(["age", "job"])
df2.show()
print(df2.count())

#Note
# df2 = df.dropDuplicates() is the same as df2 = df.distinct()

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

7
+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
| Rita| -2|Data Scientist|  1600|  500|
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

5


In [24]:
# Select rows based on a condition

df = spark.read.csv("data/people.csv", header=True)
df.show()

df2 = df.filter(df.age > 27)

# Alternative:
#df2 = df[df.age >= 27]

# With condition involving more than one column
df2 = df[(df.age > 27) & (df.job == 'Data Scientist')]

df2.show()

# We could also have a condition that compares the values of two columns
#df2 = df.filter(df.col1 > df.col2)

#Exercise: How can you remove all the rows with some specific value in a column?


+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

+----+---+--------------+------+-----+
|name|age|           job|salary|prize|
+----+---+--------------+------+-----+
|João| 30|Data Scientist|  1500|  250|
+----+---+--------------+------+-----+



In [29]:
# Remove columns with the same value for all rows and columns with different values for all rows

import pyspark.sql.functions as f

df = spark.read.csv("data/people.csv", header=True)
df.show()

cnt = df.agg(*(f.countDistinct(c).alias(c) for c in df.columns)).first()
print(cnt)

# drop columns with the same value for all rows
df.drop(*(c for c in cnt.asDict() if cnt[c] == 1)).show()

# drop columns with different values for all rows
df.drop(*(c for c in cnt.asDict() if cnt[c] == df.count())).show()

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

Row(name=5, age=4, job=3, salary=5, prize=5)
+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

+---+--------------+
|age|           job|
+---+--------------+
| 25|       Manager|
| 30|Data Scientist|
| 30|     Developer|
| -2|Data Scientist|
| 32|     Developer|
+---+--------------+



In [35]:
# Creating/changing values

from pyspark.sql.functions import when
from pyspark.sql.types import IntegerType

df = spark.read.csv("data/people.csv", header=True)
df.show()
#df.printSchema()

#df2 = df.withColumn("Yearly Income", df.salary * 14 + df.prize)

#df2 = df.withColumn("salary", df.salary.cast(IntegerType()))

#df2 = df.withColumn("job", 
#            when(df.job == "Developer","Programmer").
#            when(df.job == "Manager","Project Manager").
#            otherwise(df.job))

df2 = df.withColumn("prize", when(df.prize < 0, 0).otherwise(df.prize))

df2.show()
#df2.printSchema()


+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|   -1|
+-----+---+--------------+------+-----+

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| João| 30|Data Scientist|  1500|  250|
|Jorge| 30|     Developer|  1400|  200|
| Rita| -2|Data Scientist|  1600|  500|
|  Bob| 32|     Developer|  1000|    0|
+-----+---+--------------+------+-----+



Working with SQL

In [36]:
df = spark.read.csv("data/people.csv", header=True)

df.createOrReplaceTempView("people")

vintagesDF = spark.sql("SELECT * FROM people WHERE age < 30")
vintagesDF.show()

# See also createGlobalTempView()

+-----+---+--------------+------+-----+
| name|age|           job|salary|prize|
+-----+---+--------------+------+-----+
|Maria| 25|       Manager|  2000|  100|
| Rita| -2|Data Scientist|  1600|  500|
+-----+---+--------------+------+-----+



In [30]:
# Run SQL on files directly

df = spark.sql("SELECT * FROM parquet.`data/users.parquet`")

df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



Writing a data frame to a file

In [37]:
# Select only the "name" and "age" columns

df1 = spark.read.csv("data/people.csv", header=True)

df2 = df1.select("name", "age")

df2.show()

print(df2.rdd.getNumPartitions())

df2.write.csv("data/people_name_age", header=True)

# Alternative
#df2.write.format("csv").save("data/people_name_age")

+-----+---+
| name|age|
+-----+---+
|Maria| 25|
| João| 30|
|Jorge| 30|
| Rita| -2|
|  Bob| 32|
+-----+---+

1


Working with text files

In [38]:
df = spark.read.text("data/hello.txt")

df.show()
df.collect()

+---------------+
|          value|
+---------------+
|    hello world|
|      Hello spa|
|    hello spark|
|    spark hello|
|hellosparkhello|
+---------------+



[Row(value='hello world'),
 Row(value='Hello spa'),
 Row(value='hello spark'),
 Row(value='spark hello'),
 Row(value='hellosparkhello')]

In [36]:
# Lines containing some word

df = spark.read.text("data/hello.txt")

linesWithSpark = df.filter(df.value.contains("spark"))

linesWithSpark.show()

df.filter(df.value.contains("spark")).count()

+---------------+
|          value|
+---------------+
|    hello spark|
|    spark hello|
|hellosparkhello|
+---------------+



3

In [46]:
# Maximum row size

from pyspark.sql.functions import *

df = spark.read.text("data/hello.txt")

#df.select(size(split(df.value, "\s+")) \
#        .name("numWords")) \
#        .agg(max(col("numWords"))) \
#        .collect()

#df.select(split(df.value, "\s+").alias("wordlist")).show()

#df.select(size(split(df.value, "\s+"))).show()

#df.select(size(split(df.value, "\s+")).name("numWords")).agg(max(col("numWords"))).first().__getitem__("max(numWords)")

df.select(size(split(df.value, "\s+")).name("numWords")).agg(max(col("numWords")).name("max")).first().__getitem__("max")

# This first line maps a line to an integer value and aliases it as “numWords”, creating a new DataFrame. agg is called on that DataFrame to find the largest word count. The arguments to select and agg are both Column, we can use df.colName to get a column from a DataFrame. We can also import pyspark.sql.functions, which provides a lot of convenient functions to build a new Column from an old one.


3

In [47]:
# Number of occurences of each word

df = spark.read.text("data/hello.txt")

wordCounts = df.select(explode(split(df.value, "\s+")).alias("word")).groupBy("word").count()

wordCounts.show()

# explode(): returns a new row for each element in the given array or map. Uses the default column name col for elements in the array and key and value for elements in the map unless specified otherwise.

# Here, we use the explode function in select, to transform a Dataset of lines to a Dataset of words, and then combine groupBy and count to compute the per-word counts in the file as a DataFrame of 2 columns: “word” and “count”.

# Notice that the above code basically implements a MapReduce flow

+---------------+-----+
|           word|count|
+---------------+-----+
|          hello|    3|
|          asdsf|    1|
|hellosparkhello|    1|
|          Hello|    1|
|          spark|    2|
|          world|    1|
|            spa|    1|
+---------------+-----+



Interoperating with RDDs

In [53]:
from pyspark.sql import Row
from pyspark.sql.functions import *

sc = spark.sparkContext

rdd = sc.textFile("data/logs.txt")

#rdd.foreach(print)

# Creates a DataFrame having a single column named "line"
df = rdd.map(lambda r: Row(r)).toDF(["line"])

#df.show()

#errorsDF = df.filter(col("line").like("%ERROR%"))
#errorsDF = df.filter(col("line").like("ERROR%"))
errorsDF = df.filter(col("line").like("ERROR_%MySQL%"))

# Counts all the errors
#errorsDF.count()

# Counts errors mentioning MySQL
#errorsDF.filter(col("line").like("%MySQL%")).count()

# Fetches the MySQL errors as an array of strings
#errorsDF.filter(col("line").like("%MySQL%")).collect()

errorsDF.show()

+--------------------+
|                line|
+--------------------+
|ERROR ddfdd MySQL...|
+--------------------+



In [53]:
# Spark SQL can convert an RDD of Row objects to a DataFrame, inferring the datatypes.

from pyspark.sql import Row

def schema_inference_example(spark: SparkSession, filePath) -> None:
  
    sc = spark.sparkContext

    # Load a text file and convert each line to a Row.
    
    linesRDD = sc.textFile(filePath)

    #linesRDD.foreach(print)

    partsRDD = linesRDD.map(lambda line: line.split(","))

    #partsRDD.foreach(print)

    peopleRDD = partsRDD.map(lambda p: Row(name=p[0], age=int(p[1])))

    #peopleRDD.foreach(print)

    # Infer the schema, and register the DataFrame as a table.
    
    peopleDF = spark.createDataFrame(peopleRDD)
    
    #peopleDF.show()

    #peopleDF.printSchema()

    peopleDF.createOrReplaceTempView("people")
    
    # SQL can be run over DataFrames that have been registered as a table.
    # The results of SQL queries are Dataframe objects.

    almost30DF = spark.sql("SELECT name FROM people WHERE age >= 28 AND age < 30")

    #almost30DF.show()
    
    # rdd (below) returns the content as an :class:`pyspark.RDD` of :class:`Row`.
    # That is, it returns the content of the DataFrame as a pyspark.RDD of Row.
    
    almost30NamesRDD = almost30DF.rdd.map(lambda p: "Name: " + p.name)
    
    #almost30NamesRDD.foreach(print)

    #Showing results another way
    #arrayAlmost30 = almost30NamesRDD.collect()

    #for name in arrayAlmost30:
    #    print(name)

schema_inference_example(spark, "data/people.txt")

In [73]:
# Programmatically Specifying the Schema

from pyspark.sql.types import StringType, IntegerType, StructType, StructField

def programmatic_schema_example(spark: SparkSession, filePath) -> None:
    
    sc = spark.sparkContext

    # Load a text file and convert each line to a Row.
    linesRDD = sc.textFile(filePath)
    partsRDD = linesRDD.map(lambda line: line.split(","))
    
    # Each line is converted to a tuple.
    peopleRDD = partsRDD.map(lambda p: (p[0], p[1].strip()))

    #peopleRDD.foreach(print)

    # The schema is encoded in a string.
    schemaString = "name age"

    fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
    
    schema = StructType(fields)

    # Apply the schema to the RDD.
    peopleDF = spark.createDataFrame(peopleRDD, schema)
    peopleDF.show()

    peopleDF.printSchema()

    # Creates a temporary view using the DataFrame
    peopleDF.createOrReplaceTempView("people")

    # SQL can be run over DataFrames that have been registered as a table.
    #resultsDF = spark.sql("SELECT name FROM people")

    #resultsDF.show()

programmatic_schema_example(spark, "data/people.txt")

+-----+---+
| name|age|
+-----+---+
|Maria| 25|
| João| 29|
|Jorge| 30|
| Rita| 28|
|  Bob| 32|
+-----+---+

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



DataFrame column methods

Methods that take column names as arguments:
- corr(col1, col2): two column names.
- cov(col1, col2): two column names.
- crosstab(col1, col2): two column names.
- describe(*cols): **cols refers to only column names (strings).*

Methods that take column names or column expressions or both as arguments:
- drop(*cols): *a list of column names OR a single column expression.*
- groupBy(*cols): column name (string) or column expression or both.
- rollup(*cols): column name (string) or column expression or both.
- select(*cols): column name (string) or column expression or both.
- sort(*cols, **kwargs): column name (string) or column expression or both.
- sortWithinPartitions(*cols, **kwargs): column name (string) or column expression or both.
- orderBy(*cols, **kwargs): column name (string) or column expression or both.
- sampleBy(col, fractions, sed=None): a column name.
- toDF(*cols): a list of column names (string).
- withColumn(colName, col): colName refers to column name; col refers to a column expression.
- withColumnRenamed(existing, new): takes column names as arguments.
- filter(condition): *condition refers to a column expression that returns types.BooleanType of values.

Bibliography:
- https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#datasets-and-dataframes
- https://spark.apache.org/docs/latest/quick-start.html
- https://spark.apache.org/docs/3.1.3/api/python/ 
- Learning Spark Lightning-Fast Big Data Analysis, Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia. O'Reilley, 2015.
- https://sparkbyexamples.com/